In [10]:
# Chapter 1 Switching,  Lab 1 Configuring Trunks
# This script will help accelerate the labs in the lab book of the 
# CCIE Enterprise Infrastructure Foundations book
# This lab will use the included lab for Eveng

In [1]:
# we'll have to do some up front work to connect to the lab and do some stuff
# the default telnetlib is being depreciated and after trying the called out replacements i just built my own.... with blackjack and hookers
import socket
import selectors
import time

class Telnet:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.sock = socket.create_connection((host, port))
        self.selector = selectors.DefaultSelector()
        self.selector.register(self.sock, selectors.EVENT_READ)

    def send_command(self, command):
        self.sock.sendall(command.encode('utf-8'))

    def receive_response(self):
        data = self.sock.recv(65536)
        return data

    def receive_response_all(self):
        self.sock.setblocking()

    def close(self):
        self.sock.close()

class CiscoTelnet:
    def __init__(self, host, port, username, password):
        self.telnet = Telnet(host, port)
        self.username = username
        self.password = password

    def prompt_identifier(self):
        raw_prompt = self._read_raw_prompt()
        self._send_command('\r\n')
        time.sleep(1)
        raw_prompt = self._read_raw_prompt()
        # grab the new prompt
        prompt = raw_prompt.decode('utf-8')
        return prompt

    def _send_command(self, command):
        self.telnet.send_command(command + '\r\n')

    def _read_raw_prompt(self):
        return self.telnet.receive_response()

    def login(self):
        prompt = self.prompt_identifier()
        retry_counter = 0
        while "#" not in prompt and retry_counter < 5:
            retry_counter += 1
            if "initial" in prompt.lower() or "yes/no" in prompt.lower():
                self._send_command("no")
                time.sleep(30)
                self.prompt_identifier()  # Update prompt after 'no' command

            elif "username" in prompt.lower():
                self._send_command(self.username)
                time.sleep(1)
                self._send_command(self.password)
                self.prompt_identifier()  # Update prompt after login

            elif ">" in prompt:
                self._send_command("en")
                time.sleep(1)
                prompt = self.prompt_identifier()  # Update prompt after 'en' command

    def cisco_send_command(self, command):
        self._send_command(command)
        prompt = self.prompt_identifier()
        time.sleep(1)
        return prompt

    def close(self):
        self.telnet.close()
        
    def blackjack(self):
        return "Read it and weep"
    
    def hookers(self):
        return "You know it baby"

# # Example usage
#     host = "172.24.177.49" # eveng IP
#     port = 32769  # port for device being interacted w/, we'll do a list and go through them
#     username = "" # blank if no console username/ auth is set
#     password = "" # blank if no console password/ auth is set
#     commands = ["show version", "show interfaces"] # example list

#     cisco_napalm(host, username, password, commands, port)


In [3]:
# set the following variables for device ports and your eveng server
devices_dict = {
    "sw1_port": 32777,
    "sw2_port": 32778,
    "sw3_port": 32779,
    "sw4_port": 32780,
    "sw5_port": 32781,
    "sw6_port": 32782}

# run task 1, pg 4 commands
commands = ["en",
            "conf t",
            "interface range e0/0-3,e1/0-3,e2/0-3",
            "shut",
            "vtp domain TST",
            "interface range e0/0-3,e1/0-3,e2/0-3",
            "shut",
            "interface range e3/0-3,e4/0-3,e5/0-3,e6/0-3",
            "shut",
            "vtp domain TST",
            "end",
            "exit"
            ]
# setup the telnet session
eveng_host = "172.24.177.49"
username = "" # this is the username on the switch in the lab, not eveng itself
password = "" # same as username
# run the commands
for device, port in devices_dict.items():
    print(f"Device: {device}, Port {port} commands being send")
    cisco_device = CiscoTelnet(eveng_host, port, username, password)
    cisco_device.login()
    for command in commands:
        output = cisco_device.cisco_send_command(command)
        print(f"Command output for {device[0:3]} command {command}: \n{output}")
    cisco_device.close()

Device: sw1_port, Port 32777 commands being send
Command output for sw1 command en: 
n
Switch#
Switch#
Switch#
Switch#
Switch#
Switch#
Command output for sw1 command conf t: 
onf t
Enter configuration commands, one per line.  End with CNTL/Z.
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Command output for sw1 command interface range e0/0-3,e1/0-3,e2/0-3: 
nterface range e0/0-3,e1/0-3,e2/0-3
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Command output for sw1 command shut: 
hut
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Switch(config-if-range)#
Command output for sw1 command vtp domain TST: 
tp domain TST
Changing VTP domain name from NULL to TST
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
Switch(config)#
*Jan 25 15:04

KeyboardInterrupt: 

In [ ]:
# wait for the cell above to finish, its very verbose and to remove verbosity comment out the 
# print(f"Command output for {device[0:3]} command {command}: \n{output}") line
# if anything doesn't send rerun and fail back to manually loading
# telnet module is not the best but seems reliable enough for these purposes, usually would just bootstrap to a oobm like network then ssh reliably but with lab trying to not modify the topology and all that
